# Decission tree

Entropy:
$$ E = -∑ p(X) * log_{2} (p(X)) $$
$$ p(X) = \frac{\#x}{n} $$

Example:
$$ S = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1] $$
$$ E = - \frac{5}{10} * log_{2}(\frac{5}{10}) - \frac{5}{10} * log_{2}(\frac{5}{10}) = -0.5log_{2}(-0.5)-0.5log_{2}(-0.5) = 1 $$
$$ E = -0.5 * (-1) - 0.5 * (-1) = 1 $$

Information gain:
$$ IG = E(parent) - [weighted \ average] * E(children) $$

Example:
$$ S = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1] $$
$$ S1 = [0, 0, 1, 1, 1, 1, 1] $$
$$ S2 = [0, 0, 0] $$
$$ IG = E(S) - [\frac{7}{10} * E(S1) + \frac{3}{10} * E(S2)] $$
$$ IG = 1 - [\frac{7}{10} * 0.863 + \frac{3}{10} * 0] = 0.395 $$

Approach:
Train algorithm := Build the tree
• Start at the top node and at each node select the best split based on the best information gain.
• Greedy search: Loop over all features and over all thresholds (all possible feature values).
Save the best split feature and split threshold at each node.
• Build the tree recursively.
• Apply some stopping criteria to stop growing
e.g. here: maximum depth, minimum samples at node, no more class distribution in node.
• When we have a leaf node, store the most common class label of this node


Predict := Traverse tree
• Traverse the treelrecursively.
• At each node look at the best split feature of the test feature vector x and go left or right
depending on [feature_idx] <= threshold
• When we reach the leaf node we return the stored most common class label

In machine learning, entropy is often used as a measure of the uncertainty or randomness of a system or dataset. In this context, entropy is often used in the context of decision trees, where it is used to measure the impurity of a set of data.

In a decision tree, each internal node represents a test on an attribute, and each leaf node represents a class label. The entropy of a node is a measure of the impurity of the data at that node, with a low entropy indicating that the data is pure (i.e., all of the instances at that node belong to the same class) and a high entropy indicating that the data is impure (i.e., the instances at that node belong to multiple classes).

Entropy is used in decision tree learning to determine the best attribute to split on at each node. The attribute with the highest information gain (i.e., the attribute that reduces entropy the most) is chosen as the splitting attribute.

Entropy can also be used in other machine learning algorithms, such as clustering and feature selection, to measure the randomness or disorder of a dataset. In these contexts, entropy is used to measure the quality of a clustering or the importance of a feature in predicting the target variable.

In [1]:
from collections import Counter
import numpy as np

## clean version

In [3]:
def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])


class Node:
    def __init__(
        self, feature=None, threshold=None, left=None, right=None, *, value=None
    ):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (
            depth >= self.max_depth
            or n_labels == 1
            or n_samples < self.min_samples_split
        ):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)

        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

In [4]:
if __name__ == "__main__":
    # Imports
    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    def accuracy(y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

    data = datasets.load_breast_cancer()
    X, y = data.data, data.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1234
    )

    clf = DecisionTree(max_depth=10)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    acc = accuracy(y_test, y_pred)

    print("Accuracy:", acc)

Accuracy: 0.9385964912280702


## sklearn

In [15]:
from sklearn import tree
from sklearn import datasets
from sklearn.model_selection import train_test_split

data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)


# iris = load_iris()
# X, y = iris.data, iris.target
clf = tree.DecisionTreeClassifier(max_depth=10, min_samples_split=2)
clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)

print("Accuracy:", np.sum(y_test == y_pred) / len(y_test))

Accuracy: 0.9122807017543859
